In [1]:
import numpy as np
import pandas as pd

NODE_NUMBER = 0


class Node:
    """
    The class of node in tree, it has three instance attributes.
    is_leaf: denotes if the node is a leaf node.
    node_value: the value of the node.
    sub_node_list: contains key-value list for the sub-node of current node. 
                   key denotes the label of the sub-node, value denotes the address of sub-node.

    """
    def __init__(self):
        global NODE_NUMBER
        NODE_NUMBER = NODE_NUMBER + 1
        self.is_leaf = False
        self.node_value = ''
        self.sub_node_list = {}
        self.id = NODE_NUMBER


def cal_feature(df, target_label):
    """ Calculate the feature list.
    
    :param df: the data frame to handle
    :param target_label: the index of target 
    :return feature_list: the list of features
    """
    feature_list = list(df.columns)
    feature_list.remove(target_label)
    
    return feature_list


def calc_ent(df, target_label):
    """ Calculate the entropy of df.
    
    :param df: the data frame to handle
    :param target_label: the index of target
    :return ent: the entropy of s
    """
    df_value_list = set()
    
    for index, row in df.iterrows():
        df_value_list.add(row[target_label])
        
    ent = 0.0
    for df_value in df_value_list:
        p = float(df[df.loc[:, target_label] == df_value].shape[0])/df.shape[0]
        log_p = np.log2(p)
        ent -= p * log_p 
        
    return ent


# Calculate the information gain Gain(df, a, target_label).  
def calc_info_gain(df, a, target_label):
    """ Calculate the information gain Gain(df, a, target_label).
    
    :param df: the data frame to calculate
    :param a: the attribute
    :param target_label: the index name of target
    :return: the information gain of df by the given attribute a
    """
    
    # entropy of df
    ent_df = calc_ent(df, target_label)
    
    # a_value_list
    a_value_list = set()
    for index, row in df.iterrows():
        a_value_list.add(row[a])
    
    # Calculate the information gain of attribute a.
    info_gain = ent_df
    for a_value in a_value_list:
        df_a_value = df[df.loc[:,a] == a_value]
        ent_df_a_value = calc_ent(df_a_value, target_label)
        info_gain -= float(df_a_value.shape[0]) / df.shape[0] * ent_df_a_value
        
    return info_gain


def choose_best_attribute(df, features, target_label):
    """ Choose the best attribute according to the information gain.
    
    :param df: the data frame to handle
    :param features: feature list
    :param target_label: the index of target
    :return best_feature: the best attribute chosen
    """
    best_info_gain = 0
    best_feature = ''
    
    for attribute in features:
        info_gain = calc_info_gain(df, attribute, target_label)
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = attribute

    return best_feature


def split_data(df, feature, feature_value):
    """ Split the data by the feature value.
    
    :param df: the data frame to handle
    :param feature: the feature chosen to split the data
    :param feature_value: the value of the feature
    :return: the data with the specified feature value
    """
    new_df = df[df.loc[:, feature] == feature_value]
    return new_df


# Record all the possible value of all features.
def calculate_possible_value(df, attributes):
    """ Record all the possible value of all features.
    
    :param df: the data frame to handle
    :param attributes: attribute list
    :return: a dictionary with the features and its possible values
    """
    attribute_value_dictionary = {}
    for attribute in attributes:
        possible_value = df[attribute].drop_duplicates()
        attribute_value_dictionary[attribute] = list(possible_value)
    return attribute_value_dictionary


def test_same_value(df, target_label):
    """ Test if the observations have the same targets.
    
    :param df: the data frame to handle
    :param target_label: the index of target
    :return: the target value without duplication
    """
    data = df[target_label].drop_duplicates()
    return data
        

def create_tree(df, target_label, attributes):
    """ Generate a decesion tree given by df, target_label, attributes.
    
    :param df: the data frame to handle
    :param target_label: the index of target
    :param attributes: attribute list
    :return: a node
    """
    # Create a node.
    node = Node()
    
    # Get the target value without duplication.
    df_target = test_same_value(df, target_label)
    
    if len(df_target) == 1:
        # case 1: the observations have the same target value.
        node.node_value = df_target.values[0]
        node.is_leaf = True
    elif len(attributes) == 0:
        # case 2: there are no remained attributes.
        node.node_value = df[target_label].mode().iloc[0]
        node.is_leaf = True
    else:
        # other case: choose the best attribute as node
        best_attribute = choose_best_attribute(df, attributes, target_label) 
        node.node_value = best_attribute 
        node.is_leaf = False
        best_attribute_value_list = dictionary[best_attribute] 
        attributes.remove(best_attribute)
        
        # For each attribute value, generate the sub branch.
        for attribute_value in best_attribute_value_list:
            # Split the data by the attribute value.
            sub_df = split_data(df, best_attribute, attribute_value)
            if len(sub_df) == 0:
                # Sub-branch has no observations.
                sub_node = Node()
                sub_node.is_leaf = True
                sub_node.node_value = df[target_label].mode().iloc[0]
                node.sub_node_list[attribute_value] = sub_node   
            else:
                # Sub-branch has observations.
                new_attributes = attributes.copy() 
                node.sub_node_list[attribute_value] = create_tree(sub_df, target_label, new_attributes)
    return node


def print_tree(node, label=None, parent=None):
    """ Print the information of the tree.
    
    :param node: the node to print
    :param label: the label of the node
    :param parent: the parent of the node
    :return:
    """
    print('Node', node.id, ': ', node.node_value)
    if (label is not None) and (parent is not None):
        print('Label: ', label, '\nParent: Node', parent.id, parent.node_value, '\n')
    else:
        print('Label: None \nParent: None\n')
        
    if len(node.sub_node_list) == 0:
        return
    else:
        for label, sub_node in node.sub_node_list.items():
            print_tree(sub_node, label, node)
 
            
# Use the titanic_passenger_survival data. 
df = pd.read_csv('titanic_passenger_survival.csv', header=0, usecols=[0, 1, 2, 3]) 
target_label = 'Survived'
                    
# Use the tennis data.        
# df = pd.read_csv('tennis.csv', header=0, usecols=[1,2,3,4,5])   
# target_label = 'PlayTennis'

features = cal_feature(df, target_label)    # Obtain the feature list
dictionary = calculate_possible_value(df, features)     # Get all the possible value of features.

# Generate the tree using ID3 algorithm.
tree = create_tree(df, target_label, features)

# Print the tree.
print_tree(tree)


Node 1 :  Sex
Label: None 
Parent: None

Node 2 :  Class
Label:  Male 
Parent: Node 1 Sex 

Node 3 :  Age
Label:  3rd 
Parent: Node 2 Class 

Node 4 :  No
Label:  Child 
Parent: Node 3 Age 

Node 5 :  No
Label:  Adult 
Parent: Node 3 Age 

Node 6 :  Age
Label:  1st 
Parent: Node 2 Class 

Node 7 :  Yes
Label:  Child 
Parent: Node 6 Age 

Node 8 :  No
Label:  Adult 
Parent: Node 6 Age 

Node 9 :  Age
Label:  2nd 
Parent: Node 2 Class 

Node 10 :  Yes
Label:  Child 
Parent: Node 9 Age 

Node 11 :  No
Label:  Adult 
Parent: Node 9 Age 

Node 12 :  Class
Label:  Female 
Parent: Node 1 Sex 

Node 13 :  Age
Label:  3rd 
Parent: Node 12 Class 

Node 14 :  No
Label:  Child 
Parent: Node 13 Age 

Node 15 :  No
Label:  Adult 
Parent: Node 13 Age 

Node 16 :  Age
Label:  1st 
Parent: Node 12 Class 

Node 17 :  Yes
Label:  Child 
Parent: Node 16 Age 

Node 18 :  Yes
Label:  Adult 
Parent: Node 16 Age 

Node 19 :  Age
Label:  2nd 
Parent: Node 12 Class 

Node 20 :  Yes
Label:  Child 
Parent: Node 1